In [33]:
import io, json, codecs, os, fix

In [34]:
def redif2jsonld(jsonld_src,journal_name,issn):
    global articles; articles=[]
    global article; article={}
    global creators; creators=[]
    global creator; creator=''
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(creator)>0: creators.append(creator); creator=''
                article['ex:creator']=creators
                creators=[]
                if fix.validate(article)==True: articles.append(article)
                else: e=e+1
            article={}
            article['ex:creator']=[]
            article['ex:keyword']=[]
            article['ex:source']=journal_name
            article['ex:date']=''
            article['ex:title']=''
        elif key=='author-name':
            if len(creator)>0: creators.append(creator)
            creator=''
            if ',' in j: j=' '.join(reversed([part.strip() for part in j.split(',')]))
            creator=j
        elif key=='title':article['ex:title']=j
        elif key=='year':article['ex:date']=j
        elif key=='pages':article['ex:pages']=j
        elif key=='volume':article['ex:volume']=j
        elif key=='issue':article['ex:issue']=j
        elif key=='file-url':article['ex:url']=j
        elif key=='abstract':article['ex:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='': article['ex:keyword'].append(fix.fix_keyword(keyword))

    if len(article)>0:
        if len(creator)>0: creators.append(creator); creator=''
        article['ex:creator']=creators; creators=[]
        articles.append(article)   
        
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_folder+issn+'.jsonld'
    if not os.path.exists(jsonld_file):
        with io.open(jsonld_file, 'w', encoding="utf-8") as f:
            f.write(jd)
    return (jsonld_file)

In [35]:
redif_folder='redif/'
jsonld_src='jsonld_src/'

    
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
print len(journals)
    
for journal in journals:
    source=journal['jname']
    if source != 'jrnmis' : continue
    journal_name=journal['journal']
    data=journal['data']
    if 'issn' in journal: issn=journal['issn']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='redif' or file_type=='parse':
            if file_type=='parse': publisher='src'
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(redif_folder+publisher+'/'+source):
                if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                    redif_file=publisher+'/'+source+'/'+file_name
                    content = []
                    articles=[]
                    with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                        for line in file:
                            line = line.strip()
                            content.append(line)
                    jsonld_file=redif2jsonld(jsonld_src,journal_name,issn)
                i=i+1
            print ', INVALID=',e,'files=',i

40
src jrnmis Journal of Management Information Systems , INVALID= 1 files= 1


In [36]:
jsonld_src='jsonld_src/'
jsonld_folder='jsonld/'
with open(jsonld_src+'src_jrnmis_taf.jsonld') as data_file:    
    data = json.load(data_file)


In [37]:
data[0]

{u'ex:abstract': u'The Journal of Management Information Systems will provide an integrated view of the entire discipline to both researchers and practitioners and will help communicate research and practical advances.In order to define the focal points of the new journal, the developmental trends in mis as a field of practice and as an academic discipline are analyzed. Support of less structured decisions under the condition of organizational dynamics is found to be of particular importance. At the same time, the environment of information delivery will be determined by cooperative computing, with the end users gaining in a large measure direct access to information.The role of mis as a crucial component in the evolving knowledge society is stressed. In particular, flexibility of information systems is a factor of social productivity at large.',
 u'ex:creator': [u'Vladimir Zwass'],
 u'ex:date': u'1984',
 u'ex:issue': u'1',
 u'ex:keyword': [u'Evolution of management information systems